In [3]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#####################################################################################################################
#                                                                                                                   #
#                                              Specifing Input Values                                               #
#                                                                                                                   #
#####################################################################################################################

#engine parameters, based on Jumo205D

bore = 0.105 # [m]
stroke = 0.16 # [m]
disp_vol = 1.386e-3 # [m^3]
cmpr_ratio = 17. #[-]
n_rev = 3000.0 / 60.0 # [1/s]

# outlet pressure
p_outlet = 1.2e5  # Pa

# ambient properties
T_ambient = 293.15  # K
p_ambient = ct.one_atm  # Pa
comp_ambient = 'O2:1, N2:3.76'

# turbocharger temperature, pressure, and composition
T_inlet = 300.  # K
p_inlet = 1.3e5  # Pa
comp_inlet = 'O2:1, N2:3.76'


#POCZYTAJ O WSPÓŁOTWARCIU

# Inlet valve friction coefficient, open and close timings 
inlet_valve_coeff = 1.e-6
inlet_open = -18. / 180. * np.pi
inlet_close = 198. / 180. * np.pi

# Outlet valve friction coefficient, open and close timings
outlet_valve_coeff = 1.e-6
outlet_open = 522. / 180 * np.pi
outlet_close = 18. / 180. * np.pi

# Fuel mass, injector open and close timings
injector_open = 350. / 180. * np.pi
injector_close = 365. / 180. * np.pi
injector_mass = 3.2e-5  # kg
injector_t_open = (injector_close - injector_open) / 2. / np.pi / n_rev

# Simulation time and resolution
sim_n_revolutions = 10.0
sim_n_timesteps = 100000.0

#####################################################################################################################
#                                                                                                                   #
#                                              Setting the enviroment up                                            #
#                                                                                                                   #
#####################################################################################################################

# define gas using reaction mechanism GRI-Mech 3.0

gas = ct.Solution('gri30.xml')

#let user choose a mixture

fuel = input("Choose fuel (methane/ethane/propane): ")

if fuel == "methane":
    gas.TPX = 1000, ct.one_atm, 'CH4:1'
elif fuel == "ethane":
    ethane.TPX = 1000, ct.one_atm, 'C2H6:1'
elif fuel == "propane":
    ethane.TPX = 1000, ct.one_atm, 'C3H8:1'
else:
    print("Specified fuel not available. Write 'methane', 'ethane' or 'propane'.")
    
# fuel properties (gaseous!)
T_injector = 293.15  # K
p_injector = 551.58e5  # Pa
comp_injector = gas.X

# define initial state
gas.TPX = T_inlet, p_inlet, comp_inlet
r = ct.IdealGasReactor(gas)
# define inlet state
gas.TPX = T_inlet, p_inlet, comp_inlet
inlet = ct.Reservoir(gas)
# define injector state (gaseous!)
gas.TPX = T_injector, p_injector, comp_injector
injector = ct.Reservoir(gas)
# define outlet pressure (temperature and composition don't matter)
gas.TPX = T_ambient, p_outlet, comp_ambient
outlet = ct.Reservoir(gas)
# define ambient pressure (temperature and composition don't matter)
gas.TPX = T_ambient, p_ambient, comp_ambient
ambient_air = ct.Reservoir(gas)

# set up connecting devices
inlet_valve = ct.Valve(inlet, r)
injector_mfc = ct.MassFlowController(injector, r)
outlet_valve = ct.Valve(r, outlet)
piston = ct.Wall(ambient_air, r)


#Prepare the simulation with a ReactorNet object
sim = ct.ReactorNet([r])
time = 0.e-1

#Arrays to hold the data

times = np.zeros(400)
rtemp = np.zeros(400)
data = np.zeros((400,4))

#Advance the simulation in time and print the internal evolution of
#temperature, volume and internal energy

print('%10s %10s %10s %14s' % ('t [s]','T [K]','vol [m3]','u [J/kg]'))
for n in range(400):
    time += 5.e-3
    sim.advance(time)
    times[n] = time # time in s
    rtemp[n]=r.T
    data[n,0]=r.T
    data[n,1:]=r.thermo['O2','H','CH4'].X
    print('%10.3f %10.3f %10.3f %14.6e' % (sim.time, r.T, r.thermo.v, r.thermo.u))
    
#Show graphs of T [K] against time
plt.plot(times, rtemp)
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
plt.show()

Choose fuel (methane/ethane/propane): methane
     t [s]      T [K]   vol [m3]       u [J/kg]


CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::integrate:
CVodes error encountered. Error code: -9
The right-hand side routine failed at the first call.

Exceptions caught during RHS evaluation:
Device is not ready; some parameters have not been set.
Components with largest weighted error estimates:
50: 3.65062e+11
52: 6.91709e-295
53: 6.91709e-295
33: 6.91705e-295
54: 4.68401e-295
55: 4.68401e-295
17: 2.9708e-298
8: 2.7586e-298
32: 2.122e-298
5: 2.122e-298
***********************************************************************
